In [8]:
#df processing imports
import numpy as np
import pandas as pd

#fuzzy join imports
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import textdistance

In [36]:
name_i = "/home/mrinalmanu/Desktop/analyse_pubmed_search_results/FINAL/topics/digital_twin/ready_DIG_T_DF_pre_cleaned_processed"
name_f = "/home/mrinalmanu/Desktop/analyse_pubmed_search_results/FINAL/topics/digital_twin/uni_DIG_T_DF"
big_df = "/home/mrinalmanu/Desktop/analyse_pubmed_search_results/FINAL/topics/digital_twin/uni_DIG_T_DF_bigdf"

# QS DB processing

In [3]:
# UNI rankings DF
qs_database = "/home/mrinalmanu/Desktop/analyse_pubmed_search_results/FINAL/QS-World-University-Rankings-2021-excel-table.csv"

In [5]:
# Load in the dataframe
df = pd.read_csv(qs_database, sep='/')
df = df.iloc[2:] # first 3 rows
df = df.iloc[:1000] # first 1000 rows
del df['Unnamed: 1']
to_drop = ['Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18',
       'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22']
df = df.drop(labels=to_drop, axis=1) # axis 1 drops columns, 0 will drop rows that match index value in labels
df = df.rename(columns=df.iloc[0])
df = df.iloc[1:] # first row remove
# remove all special characters everywhere

df['rank display'] = df['rank display'].str.replace('=','').str.replace(' +',' ').str.strip()

df=df.rename(columns = {'institution':'affiliation'})
df.head(1)

df_qs = df

df.head()

,rank display,affiliation,country code,country,size,focus,research,age band
3,1,Massachusetts Institute of Technology (MIT),US,United States,M,CO,VH,5
4,2,Stanford University,US,United States,L,FC,VH,5
5,3,Harvard University,US,United States,L,FC,VH,5
6,4,California Institute of Technology (Caltech),US,United States,S,CO,VH,5
7,5,University of Oxford,UK,United Kingdom,L,FC,VH,5


# Fuzzy matching the QS names to Pubmed results

In [10]:
# Load the processed data

df_n = pd.read_csv(name_i)


In [11]:
# groupings
affiliation_grouping = df_n.groupby(['affiliation']).size().reset_index(name='affiliation_count')
affiliation_grouping.head(n=1)

,affiliation,affiliation_count
0,Germany Institute of Human Nutrition PotsdamR...,1


In [12]:
author_grouping = df_n.groupby(['author']).size().reset_index(name='author_count')
author_grouping.head(n=1)

,author,author_count
0,..d..m Domonkos T..rnoki,2


In [13]:
# FUZZY JOIN

df1 = df
df2 = affiliation_grouping

# cross join the data frames
 
df1['dummy'] = True
df2['dummy'] = True
 
df = pd.merge(df1,df2, on='dummy')
 
df.drop('dummy', axis=1, inplace=True)
df.head()

,rank display,affiliation_x,country code,country,size,focus,research,age band,affiliation_y,affiliation_count
0,1,Massachusetts Institute of Technology (MIT),US,United States,M,CO,VH,5,Germany Institute of Human Nutrition PotsdamR...,1
1,1,Massachusetts Institute of Technology (MIT),US,United States,M,CO,VH,5,Healthcare Life Sciences Uppsala Sweden,2
2,1,Massachusetts Institute of Technology (MIT),US,United States,M,CO,VH,5,Vingmed Ultrasound Horton Norway,2
3,1,Massachusetts Institute of Technology (MIT),US,United States,M,CO,VH,5,ANSYS Villeurbanne F France,2
4,1,Massachusetts Institute of Technology (MIT),US,United States,M,CO,VH,5,APHP Department of Dermatology INSERM U Univer...,1


Description of ratio:

Ratio: It refers to the Levenshtein Distance Ratio.

Partial Ratio: Assume that we are dealing with two strings of different lengths such as L1 and L2, and assume that 
L1 is less than L2. Then the algorithm seeks the score of the best matching of length -L1 substring.

Token Sort Ratio: First it removes punctuations and converts the text to lower case and then it tokenizes it. Then it sorts the tokens alphabetically and then it joins them in a single string.

Token Set Ratio: Similar to the Token Sort Ratio, but it takes into consideration the unique tokens.

In [14]:
df['Ratio'] = df[['affiliation_x','affiliation_y']].apply(lambda x:fuzz.ratio(x.affiliation_x, x.affiliation_y), axis=1)
df['Partial_Ratio'] = df[['affiliation_x','affiliation_y']].apply(lambda x:fuzz.partial_ratio(x.affiliation_x, x.affiliation_y), axis=1)
df['Token_Sort_Ratio'] = df[['affiliation_x','affiliation_y']].apply(lambda x:fuzz.token_sort_ratio(x.affiliation_x, x.affiliation_y), axis=1)
df['Token_Set_Ratio'] = df[['affiliation_x','affiliation_y']].apply(lambda x:fuzz.token_set_ratio(x.affiliation_x, x.affiliation_y), axis=1)

In [15]:
df.to_csv(name_f, index=False)

In [22]:
df = pd.read_csv(name_f)

/home/mrinalmanu/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
df.head(1)

,rank display,affiliation_x,country code,country,size,focus,research,age band,affiliation_y,affiliation_count,Ratio,Partial_Ratio,Token_Sort_Ratio,Token_Set_Ratio
0,1,Massachusetts Institute of Technology (MIT),US,United States,M,CO,VH,5.0,Germany Institute of Human Nutrition PotsdamR...,1,35,42,41,46


This database will be further processed to filter false matches by Token Set Ratio.

In [26]:
df['Rank_Token_Set_Ratio'] = df.groupby('affiliation_x')['Token_Set_Ratio'].rank(ascending=False, method='dense')
df = df[['affiliation_x','affiliation_y', 'Token_Set_Ratio']].loc[df.Rank_Token_Set_Ratio==1]


There are a lot of errors with fuzzy matching approach. Most of them have to do with the fact that we have limited search to 1000 universities, and the condition that there should be a match. But we can deal with the error using a simple threshold. We can set a cutoff and filter for the mismatches. Let's look at matches for >70 Token_Set_Ratio

In [29]:
df = df[df['Token_Set_Ratio'] >= 70]
len(df)

69681

In [32]:
# merge with QS rankings DB
df_m = pd.merge(df_n, df, left_on='affiliation', right_on='affiliation_y')
df_m=df_m.rename(columns = {'affiliation_y':'original_affiliation'})
df_m=df_m.rename(columns = {'affiliation_x':'fuzzy_matched_QS'})
df_final = pd.merge(df_qs, df_m, left_on='affiliation', right_on='fuzzy_matched_QS')

In [35]:
del df_final['affiliation_x']
del df_final['name_x']
del df_final['code']
df_final.head(1)


,rank display,country code,country,size,focus,research,age band,dummy,doi,pmid,...,title,abstract,author,affiliation_y,latitude,longitude,continent,fuzzy_matched_QS,original_affiliation,Token_Set_Ratio
0,1,US,United States,M,CO,VH,5,True,10.1073/pnas.2016271118,33622790,...,Heritability of individualized cortical networ...,Human cortex is patterned by a complex and int...,Ru Kong,Stanley Center for Psychiatric Research Broad ...,37.09024,-95.712891,North America,Massachusetts Institute of Technology (MIT),Stanley Center for Psychiatric Research Broad ...,95


In [37]:
df_final.to_csv(big_df, index=False)